## Notebook to Calculate Game Goal Differntial from Kaggle and EH data
#### 2007/2008-2019/2020 are from Kaggle, 2021- is from EH via Kevin

### 2007-2019
- Open game.csv and game_team_stats.csv files and read them into a dataframes
- start by adding non-OT games to a csv with ID, home team, away team, home score, away score
- for OT games, get the game_team_stats for that game and add the same info. this will separate out SO games

### 2019-2022
- for each season, we have a csv with all the goals. 
- open the file, group it by game_id, and use groupby to get the sum of home/away goals that were not 1v0 or 0v1 gamestate (not shootout)
- add the game_id, home team, away team, home score, away score to a csv